We'll write a simplifier a la Luckasz

In [57]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq
import matplotlib.pyplot as plt
from tqdm import tqdm


class Solver:
    def __init__(self, n_qubits=3, qlr=0.01, qepochs=200,verbose=0, g=1, J=0):

        """"solver with n**2 possible actions: n(n-1) CNOTS + n 1-qubit unitary"""
        self.n_qubits = n_qubits
        self.qubits = cirq.GridQubit.rect(1, n_qubits)
        self.lower_bound_Eg = -2*self.n_qubits
        
        self.qlr = qlr
        self.qepochs=qepochs
        self.verbose=verbose


        self.indexed_cnots = {}
        self.cnots_index = {}
        count = 0
        for control in range(self.n_qubits):
            for target in range(self.n_qubits):
                if control != target:
                    self.indexed_cnots[str(count)] = [control, target]
                    self.cnots_index[str([control,target])] = count
                    count += 1
        self.number_of_cnots = len(self.indexed_cnots)
        
        self.final_params = []
        self.parametrized_unitary = [cirq.rz, cirq.rx, cirq.rz]
        
        self.observable=self.ising_obs(g=g, J=J)
        
    def ising_obs(self, g=1, J=0):
        # g \sum_i Z_i - J \sum_{i} X_i X_{i+1}
        observable = [float(g)*cirq.Z.on(q) for q in self.qubits] 
        for q in range(len(self.qubits)):
            observable.append(float(J)*cirq.X.on(self.qubits[q])*cirq.X.on(self.qubits[(q+1)%len(self.qubits)]))
        self.ground_energy = -g*np.sum(np.sqrt([1+(J/4*g)**2 - (np.cos(q)*(J/2*g)) for q in range(self.n_qubits)]))
        return observable
        
    def index_meaning(self,index):
        if index<self.number_of_cnots:
            print("cnot: ",self.indexed_cnots[str(index)])
            return
        else:
            print("1-qubit unitary on: ",(index-self.number_of_cnots)%self.n_qubits)
            return

    def append_to_circuit(self, ind, circuit, params):
        """
        appends to circuit the index of the gate;
        and if one_hot_gate it implies a rotation,
        appends to params a symbol
        """

        if ind < self.number_of_cnots:
            control, target = self.indexed_cnots[str(ind)]
            circuit.append(cirq.CNOT.on(self.qubits[control], self.qubits[target]))
            return circuit, params
        else:
            qubit = self.qubits[(ind-self.number_of_cnots)%self.n_qubits]
            for par, gate in zip(range(3),self.parametrized_unitary):
                new_param = "th_"+str(len(params))
                params.append(new_param)
                circuit.append(gate(sympy.Symbol(new_param)).on(qubit))
            return circuit, params
        
    def give_circuit(self, lista,one_hot=False):
        circuit, symbols = [], []
        for k in lista:
            circuit, symbols = self.append_to_circuit(k,circuit,symbols)
        circuit = cirq.Circuit(circuit)
        return circuit, symbols

    def dressed_cnot(self,q1,q2):
        u1 = self.number_of_cnots + q1
        u2 = self.number_of_cnots + q2
        cnot = self.cnots_index[str([q1,q2])]
        u3 = self.number_of_cnots + q1
        u4 = self.number_of_cnots + q2
        return [u1,u2,cnot,u3,u4]
    
    def dressed_ansatz(self, layers=1):
        c=[]
        for layer in range(layers):
            qubits = list(range(self.n_qubits))
            qdeph = qubits[layers:]
            for q in qubits[:layers]:
                qdeph.append(q)
            for ind1, ind2 in zip(qubits,qdeph):
                for k in self.dressed_cnot(ind1,ind2):
                    c.append(k)
        return c


    def TFQ_model(self, symbols):
        circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
        output = tfq.layers.Expectation()(
                circuit_input,
                symbol_names=symbols,
                operators=tfq.convert_to_tensor([self.observable]),
                initializer=tf.keras.initializers.RandomNormal()) #we may change this!!!
        model = tf.keras.Model(inputs=circuit_input, outputs=output)
        adam = tf.keras.optimizers.Adam(learning_rate=self.qlr)
        model.compile(optimizer=adam, loss='mse')
        return model


    def run_circuit(self, gates_index, sim_q_state=False):
        """
        takes as input vector with actions described as integer
        and outputsthe energy of that circuit (w.r.t self.observable)
        """
        ### create a vector with the gates on the corresponding qubit(s)
        circuit, symbols = self.give_circuit(gates_index)
        
        ### this is because each qubit should be "activated" in TFQ to do the optimization (if the observable has support on this qubit as well and you don't add I then error)
        effective_qubits = list(circuit.all_qubits())
        for k in self.qubits:
            if k not in effective_qubits:
                circuit.append(cirq.I.on(k))

        tfqcircuit = tfq.convert_to_tensor([circuit])
        if len(symbols) == 0:
            expval = tfq.layers.Expectation()(
                                            tfqcircuit,
                                            operators=tfq.convert_to_tensor([self.observable]))
            energy = np.float32(np.squeeze(tf.math.reduce_sum(expval, axis=-1, keepdims=True)))
            self.final_params = []

        else:
            model = self.TFQ_model(symbols)
            qoutput = tf.ones((1, 1))*self.lower_bound_Eg
            model.fit(x=tfqcircuit, y=qoutput, batch_size=1, epochs=self.qepochs, verbose=self.verbose)
            energy = np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))
            self.final_params = [np.squeeze(k.numpy()) for k in model.trainable_variables]

        #if sim_q_state:
            #simulator = cirq.Simulator()
            #result = simulator.simulate(circuit, qubit_order=self.qubits)
            #probs = np.abs(result.final_state)**2
            #return energy, probs
        return energy

In [48]:
def simplify_circuit(sol,indexed_circuit):
    """this function kills repeated unitaries and CNOTS and returns a simplified indexed_circuit vector"""
    #load circuit on each qubit
    connections={str(q):[] for q in range(sol.n_qubits)} #this saves the gates in each qubit
    places_gates = {str(q):[] for q in range(sol.n_qubits)} #this saves, for each gate on each qubit, the position in the original indexed_circuit
    

    flagged = [False]*len(indexed_circuit) #to check if you have seen a cnot already, so not to append it twice to the qubit's dictionary
    
    for q in range(sol.n_qubits): #sweep over all qubits
        for nn,idq in enumerate(indexed_circuit): #sweep over all gates in original circuit's vector
            if idq<sol.number_of_cnots: #if the gate it's a CNOT or not
                control, target = sol.indexed_cnots[str(idq)] #give control and target qubit
                if q in [control, target] and not flagged[nn]: #if the qubit we are looking at is affected by this CNOT, and we haven't add this CNOT to the dictionary yet
                    connections[str(control)].append(idq) 
                    connections[str(target)].append(idq)
                    places_gates[str(control)].append(nn)
                    places_gates[str(target)].append(nn)
                    flagged[nn] = True #so you don't add the other
            else:
                if idq%sol.n_qubits == q: #check if the unitary is applied to the qubit we are looking at
                    connections[str(q)].append("u")
                    places_gates[str(q)].append(nn)
    
       
    ### now reducing the circuit
    new_indexed_circuit = indexed_circuit.copy()
    for q, path in connections.items(): ###sweep over qubits: path is all the gates that act this qubit during the circuit 
        for ind,gate in enumerate(path):
            if gate == "u": ## IF GATE IS SINGLE QUIT UNITARY, CHECK IF THE NEXT ONES ARE ALSO UNITARIES AND KILL 'EM
                for k in range(len(path)-ind-1):
                    if path[ind+k+1]=="u":
                        new_indexed_circuit[places_gates[str(q)][ind+k+1]] = -1
                    else:
                        break
            elif gate in range(sol.number_of_cnots) and ind<len(path)-1: ### sol.number_of_cnots is the maximum index of a CNOT gate for a fixed sol.n_qubits. 
                if path[ind+1]==gate and not (new_indexed_circuit[places_gates[str(q)][ind]] == -1): #check if the next gate is the same CNOT; and check if I haven't corrected the original one (otherwise you may simplify 3 CNOTs to id)
                    others = sol.indexed_cnots[str(gate)].copy()
                    others.remove(int(q)) #the other qubit affected by the CNOT
                    for jind, jgate in enumerate(connections[str(others[0])][:-1]): ##sweep the other qubit's gates until i find "gate"
                        if jgate == gate and connections[str(others[0])][jind+1] == gate: ##i find the same gate that is repeated in both the original qubit and this one
                            if (places_gates[str(q)][ind] == places_gates[str(others[0])][jind]) and (places_gates[str(q)][ind+1] == places_gates[str(others[0])][jind+1]): #check that positions in the indexed_circuit are the same
                             ###maybe I changed before, so I have repeated in the original but one was shut down..
                                new_indexed_circuit[places_gates[str(q)][ind]] = -1 ###just kill the repeated CNOTS
                                new_indexed_circuit[places_gates[str(q)][ind+1]] = -1 ###just kill the repeated CNOTS
    final=[]
    for gmarked in new_indexed_circuit:
        if not gmarked == -1:
            final.append(gmarked)
    return final

Some examples:

In [52]:
sol=Solver()
indexed_circuit = [0,0], [0,0,0], [6,6], [0,6,0,0,6], [6,7,6,7,8,6], [6,8]#[0,1,1,0,6,7,7,8,6,8,2,7,7,6,8,6,7]
for c in indexed_circuit:
    simp = simplify_circuit(sol,c)
    print("indexed: ", c)
    print("simplified: ", simp)
    print("original: \n",sol.give_circuit(c)[0])
    print("simplified: \n",sol.give_circuit(simp)[0])
    print("\n\n")

indexed:  [0, 0]
simplified:  []
original: 
 (0, 0): ───@───@───
           │   │
(0, 1): ───X───X───
simplified: 
 



indexed:  [0, 0, 0]
simplified:  [0]
original: 
 (0, 0): ───@───@───@───
           │   │   │
(0, 1): ───X───X───X───
simplified: 
 (0, 0): ───@───
           │
(0, 1): ───X───



indexed:  [6, 6]
simplified:  [6]
original: 
 (0, 0): ───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_3)───Rx(th_4)───Rz(th_5)───
simplified: 
 (0, 0): ───Rz(th_0)───Rx(th_1)───Rz(th_2)───



indexed:  [0, 6, 0, 0, 6]
simplified:  [0, 6, 6]
original: 
 (0, 0): ───@───Rz(th_0)───Rx(th_1)───Rz(th_2)───@───@───Rz(th_3)───Rx(th_4)───Rz(th_5)───
           │                                    │   │
(0, 1): ───X────────────────────────────────────X───X────────────────────────────────────
simplified: 
 (0, 0): ───@───Rz(th_0)───Rx(th_1)───Rz(th_2)───Rz(th_3)───Rx(th_4)───Rz(th_5)───
           │
(0, 1): ───X─────────────────────────────────────────────────────────────────────



indexed:  [6, 7, 6, 7, 8,